<a href="https://colab.research.google.com/github/JiHoonPark96/practice/blob/main/econometrics_1209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf
import seaborn as sns
import matplotlib.pyplot as plt

# Load data (Upload ecls_k5.csv manually in Colab)
df = pd.read_csv("ecls_k5.csv")

# Show first rows
df.head()


,region,gender,race,bmi,mom_educ,mom_married_at_birth,family_income,mom_work_status,siblings,hhsize,pct_minority,part_dance,part_athletics,part_club,part_music,part_art,tv_afternoon_mf,tv_afterdinner_mf,tv_saturday,tv_sunday,dinner_as_family,home_language_nonenglish,both_parents,school_type,problem_crowding,problem_turnover,problem_parents,problem_drugs,problem_gangs,problem_crime,problem_weapons,problem_attacks,has_library_card,has_home_computer,school_has_security,reading_test,math_test,science_test,mom_curr_married,family_type
0,1,2,1,18.1800,3,1,120000.0000,2,0,3,1,0,1,0,1,0,1,3,1,1,6,0,1,1,0,0,0,1,1,1,0,0,0,1,0,115.6279,110.0046,112.3726,1,2
1,1,1,1,15.8800,5,1,55000.0000,1,1,4,4,0,1,1,0,0,1,0,0,0,6,0,1,1,0,0,0,0,0,0,0,0,0,1,0,103.7648,91.9275,95.1994,1,1
2,1,2,1,16.8000,5,1,70000.0000,1,1,4,1,1,1,1,1,0,0,0,0,1,7,0,1,1,0,0,0,1,1,1,0,0,0,1,0,104.0809,105.2385,104.4781,1,1
3,1,2,1,21.4700,5,1,50000.0000,1,1,4,1,1,0,1,1,0,0,1,3,3,4,0,1,1,0,0,0,1,1,1,0,0,0,1,0,103.6669,106.1771,105.4226,1,1
4,1,2,1,24.7600,3,1,50000.0000,2,2,5,1,0,1,0,0,0,0,2,4,4,7,0,1,1,0,0,0,1,1,1,0,0,0,1,0,96.0816,79.1319,96.2376,1,1


In [25]:
df = df.copy()

# Drop rows without math_test
df = df.dropna(subset=["math_test"])

# Create mom education dummy variables
df["mom_low"] = np.where(df["mom_educ"].isin([1,2]), 1, 0)
df["mom_hs"] = np.where(df["mom_educ"] == 3, 1, 0)
df["mom_somecol"] = np.where(df["mom_educ"] == 4, 1, 0)
df["mom_college"] = np.where(df["mom_educ"] == 5, 1, 0)

# Base category: mom_low (omit it from regression)
# Drop siblings due to multicollinearity
df = df.drop(columns=["siblings"])

# Log-transform income
df["log_income"] = np.log(df["family_income"] + 1)

# Create interaction term for home_language
df["lang_mom_hs"] = df["home_language_nonenglish"] * df["mom_hs"]
df["lang_mom_somecol"] = df["home_language_nonenglish"] * df["mom_somecol"]
df["lang_mom_college"] = df["home_language_nonenglish"] * df["mom_college"]

df.head()


,region,gender,race,bmi,mom_educ,mom_married_at_birth,family_income,mom_work_status,hhsize,pct_minority,part_dance,part_athletics,part_club,part_music,part_art,tv_afternoon_mf,tv_afterdinner_mf,tv_saturday,tv_sunday,dinner_as_family,home_language_nonenglish,both_parents,school_type,problem_crowding,problem_turnover,problem_parents,problem_drugs,problem_gangs,problem_crime,problem_weapons,problem_attacks,has_library_card,has_home_computer,school_has_security,reading_test,math_test,science_test,mom_curr_married,family_type,mom_low,mom_hs,mom_somecol,mom_college,log_income,lang_mom_hs,lang_mom_somecol,lang_mom_college
0,1,2,1,18.1800,3,1,120000.0000,2,3,1,0,1,0,1,0,1,3,1,1,6,0,1,1,0,0,0,1,1,1,0,0,0,1,0,115.6279,110.0046,112.3726,1,2,0,1,0,0,11.6953,0,0,0
1,1,1,1,15.8800,5,1,55000.0000,1,4,4,0,1,1,0,0,1,0,0,0,6,0,1,1,0,0,0,0,0,0,0,0,0,1,0,103.7648,91.9275,95.1994,1,1,0,0,0,1,10.9151,0,0,0
2,1,2,1,16.8000,5,1,70000.0000,1,4,1,1,1,1,1,0,0,0,0,1,7,0,1,1,0,0,0,1,1,1,0,0,0,1,0,104.0809,105.2385,104.4781,1,1,0,0,0,1,11.1563,0,0,0
3,1,2,1,21.4700,5,1,50000.0000,1,4,1,1,0,1,1,0,0,1,3,3,4,0,1,1,0,0,0,1,1,1,0,0,0,1,0,103.6669,106.1771,105.4226,1,1,0,0,0,1,10.8198,0,0,0
4,1,2,1,24.7600,3,1,50000.0000,2,5,1,0,1,0,0,0,0,2,4,4,7,0,1,1,0,0,0,1,1,1,0,0,0,1,0,96.0816,79.1319,96.2376,1,1,0,1,0,0,10.8198,0,0,0


In [26]:
corr_matrix = df[["math_test", "mom_educ", "hhsize", "log_income",
                  "home_language_nonenglish"]].corr()

corr_matrix


,math_test,mom_educ,hhsize,log_income,home_language_nonenglish
math_test,1.0000,0.3897,-0.0903,0.3473,-0.1236
mom_educ,0.3897,1.0000,-0.1399,0.4853,-0.3283
hhsize,-0.0903,-0.1399,1.0000,-0.0563,0.1971
log_income,0.3473,0.4853,-0.0563,1.0000,-0.2432
home_language_nonenglish,-0.1236,-0.3283,0.1971,-0.2432,1.0000


In [27]:
model1 = smf.ols(
    "math_test ~ mom_hs + mom_somecol + mom_college",
    data=df
).fit()

print(model1.summary())


                            OLS Regression Results                            
Dep. Variable:              math_test   R-squared:                       0.158
Model:                            OLS   Adj. R-squared:                  0.158
Method:                 Least Squares   F-statistic:                     508.5
Date:                Mon, 08 Dec 2025   Prob (F-statistic):          9.10e-303
Time:                        15:18:18   Log-Likelihood:                -29463.
No. Observations:                8105   AIC:                         5.893e+04
Df Residuals:                    8101   BIC:                         5.896e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      92.3000      0.304    303.630      

In [28]:
model2 = smf.ols(
    "math_test ~ mom_hs + mom_somecol + mom_college + log_income + hhsize + both_parents",
    data=df
).fit()

print(model2.summary())


                            OLS Regression Results                            
Dep. Variable:              math_test   R-squared:                       0.198
Model:                            OLS   Adj. R-squared:                  0.197
Method:                 Least Squares   F-statistic:                     332.5
Date:                Mon, 08 Dec 2025   Prob (F-statistic):               0.00
Time:                        15:18:25   Log-Likelihood:                -29270.
No. Observations:                8105   AIC:                         5.855e+04
Df Residuals:                    8098   BIC:                         5.860e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       75.3424      1.290     58.410   

In [29]:
home_vars = [
    "tv_afternoon_mf",
    "tv_afterdinner_mf",
    "tv_saturday",
    "tv_sunday",
    "dinner_as_family",
    "home_language_nonenglish",
    "mom_work_status",
    "mom_married_at_birth",
    "has_library_card",
    "has_home_computer"
]

formula_home = "math_test ~ mom_hs + mom_somecol + mom_college + log_income + hhsize + both_parents + " + \
                " + ".join(home_vars)

model3 = smf.ols(formula_home, data=df).fit()
print(model3.summary())


                            OLS Regression Results                            
Dep. Variable:              math_test   R-squared:                       0.225
Model:                            OLS   Adj. R-squared:                  0.223
Method:                 Least Squares   F-statistic:                     146.7
Date:                Mon, 08 Dec 2025   Prob (F-statistic):               0.00
Time:                        15:18:34   Log-Likelihood:                -29130.
No. Observations:                8105   AIC:                         5.829e+04
Df Residuals:                    8088   BIC:                         5.841e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [30]:
school_vars = [
    "school_type",
    "pct_minority",
    "school_has_security",
    "problem_crowding",
    "problem_turnover",
    "problem_parents",
    "problem_drugs",
    "problem_gangs",
    "problem_crime",
    "problem_weapons",
    "problem_attacks"
]

formula_school = formula_home + " + " + " + ".join(school_vars)

model4 = smf.ols(formula_school, data=df).fit()
print(model4.summary())


                            OLS Regression Results                            
Dep. Variable:              math_test   R-squared:                       0.235
Model:                            OLS   Adj. R-squared:                  0.232
Method:                 Least Squares   F-statistic:                     91.84
Date:                Mon, 08 Dec 2025   Prob (F-statistic):               0.00
Time:                        15:18:41   Log-Likelihood:                -29077.
No. Observations:                8105   AIC:                         5.821e+04
Df Residuals:                    8077   BIC:                         5.841e+04
Df Model:                          27                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [31]:
model5 = smf.ols(
    "math_test ~ mom_hs + mom_somecol + mom_college + "
    "log_income + hhsize + both_parents + "
    "home_language_nonenglish + lang_mom_hs + lang_mom_somecol + lang_mom_college + "
    + " + ".join(home_vars) + " + " + " + ".join(school_vars),
    data=df
).fit()

print(model5.summary())


                            OLS Regression Results                            
Dep. Variable:              math_test   R-squared:                       0.236
Model:                            OLS   Adj. R-squared:                  0.234
Method:                 Least Squares   F-statistic:                     83.35
Date:                Mon, 08 Dec 2025   Prob (F-statistic):               0.00
Time:                        15:18:49   Log-Likelihood:                -29069.
No. Observations:                8105   AIC:                         5.820e+04
Df Residuals:                    8074   BIC:                         5.842e+04
Df Model:                          30                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

In [32]:
def clean_table(model, name):
    summary = model.summary2().tables[1]
    summary["model"] = name
    return summary

table1 = clean_table(model1, "Baseline")
table2 = clean_table(model2, "Family Controls")
table3 = clean_table(model3, "Home Environment")
table4 = clean_table(model4, "School Controls")
table5 = clean_table(model5, "Interaction Model")

final_table = pd.concat([table1, table2, table3, table4, table5])
final_table


,Coef.,Std.Err.,t,P>|t|,[0.025,0.975],model
Intercept,92.3000,0.3040,303.6299,0.0000,91.7041,92.8959,Baseline
mom_hs,5.4743,0.3506,15.6120,0.0000,4.7870,6.1617,Baseline
mom_somecol,8.4355,0.3613,23.3478,0.0000,7.7273,9.1437,Baseline
mom_college,12.8681,0.3608,35.6677,0.0000,12.1609,13.5753,Baseline
Intercept,75.3424,1.2899,58.4096,0.0000,72.8139,77.8710,Family Controls
...,...,...,...,...,...,...,...
problem_drugs,-1.3751,0.3115,-4.4148,0.0000,-1.9856,-0.7645,Interaction Model
problem_gangs,-0.1266,0.3438,-0.3681,0.7128,-0.8005,0.5474,Interaction Model
problem_crime,0.6116,0.3002,2.0374,0.0416,0.0232,1.2000,Interaction Model
problem_weapons,0.0347,0.3047,0.1138,0.9094,-0.5626,0.6320,Interaction Model


In [33]:
final_table.to_csv("final_regression_results.csv", index=True)
print("Export completed: final_regression_results.csv")


Export completed: final_regression_results.csv


In [36]:
# ===== Final Comprehensive Summary Cell =====

print("\n==============================================")
print(" FINAL MODEL RESULTS SUMMARY (AUTO-GENERATED) ")
print("==============================================\n")

def extract_key_results(model, name):
    coefs = model.params
    pvals = model.pvalues

    result_text = f"\nModel: {name}\n"
    result_text += "-" * 40 + "\n"

    key_vars = [
        "mom_hs", "mom_somecol", "mom_college",
        "log_income", "hhsize", "both_parents",
        "home_language_nonenglish",
        "lang_mom_hs", "lang_mom_somecol", "lang_mom_college"
    ]

    for v in key_vars:
        if v in coefs.index:
            coef = round(coefs[v], 3)
            p = round(pvals[v], 4)
            sig = ""
            if p < 0.01: sig = "***"
            elif p < 0.05: sig = "**"
            elif p < 0.10: sig = "*"

            result_text += f"{v}: coef={coef}, p={p} {sig}\n"

    return result_text


summary_text = ""
summary_text += extract_key_results(model1, "Model 1: Baseline")
summary_text += extract_key_results(model2, "Model 2: Family Controls")
summary_text += extract_key_results(model3, "Model 3: Home Environment")
summary_text += extract_key_results(model4, "Model 4: School Controls")
summary_text += extract_key_results(model5, "Model 5: Interaction Model")

print(summary_text)


# ===== High-Level Interpretation for Report =====

print("\n==============================================")
print(" INTERPRETATION SUMMARY FOR RESEARCH PAPER ")
print("==============================================\n")

print("1. Across all models, mother’s education remains one of the strongest predictors of math performance.")
print("   Students whose mothers completed some college or a college degree score significantly higher.")

print("\n2. Log-transformed family income is consistently positive and significant,")
print("   confirming that socioeconomic status strongly contributes to academic outcomes.")

print("\n3. Household size is a relevant control without multicollinearity problems (since siblings was removed).")

print("\n4. The interaction terms between home language and mother’s education show that in non-English households,")
print("   the returns to maternal education differ, supporting the hypothesis about cognitive home environment.")

print("\n5. School environment variables (crime, gangs, weapons, etc.) introduce additional variation,")
print("   but the core family variables remain robust.")

print("\n6. The final model (Model 5) is the preferred specification for the paper,")
print("   combining family, home, and school factors with interaction effects.")




 FINAL MODEL RESULTS SUMMARY (AUTO-GENERATED) 


Model: Model 1: Baseline
----------------------------------------
mom_hs: coef=5.474, p=0.0 ***
mom_somecol: coef=8.436, p=0.0 ***
mom_college: coef=12.868, p=0.0 ***

Model: Model 2: Family Controls
----------------------------------------
mom_hs: coef=3.822, p=0.0 ***
mom_somecol: coef=6.037, p=0.0 ***
mom_college: coef=9.306, p=0.0 ***
log_income: coef=1.846, p=0.0 ***
hhsize: coef=-0.514, p=0.0 ***
both_parents: coef=2.333, p=0.0 ***

Model: Model 3: Home Environment
----------------------------------------
mom_hs: coef=3.118, p=0.0 ***
mom_somecol: coef=5.012, p=0.0 ***
mom_college: coef=7.851, p=0.0 ***
log_income: coef=1.359, p=0.0 ***
hhsize: coef=-0.479, p=0.0 ***
both_parents: coef=1.082, p=0.0002 ***
home_language_nonenglish: coef=0.299, p=0.338 

Model: Model 4: School Controls
----------------------------------------
mom_hs: coef=2.94, p=0.0 ***
mom_somecol: coef=4.837, p=0.0 ***
mom_college: coef=7.537, p=0.0 ***
log_incom